In [1]:
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

sys.path.append(os.path.abspath("../")) 

from utils.data_utils import DeepFashionSubsetDataset
from models.models import LargerShapeTextureNet
from torchvision import transforms
from torch.utils.data import DataLoader 


In [2]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
])

In [4]:
train_csv = "../data/smaller-subset/train/classification_metadata.csv"
train_img_folder = "../data/smaller-subset/train/images"

val_csv = "../data/smaller-subset/val/classification_metadata.csv"
val_img_folder = "../data/smaller-subset/val/images"

train_dataset = DeepFashionSubsetDataset(
    csv_file=train_csv,
    images_folder=train_img_folder,
    transform=train_transform,
    use_bbox=True
)

val_dataset = DeepFashionSubsetDataset(
    csv_file=val_csv,
    images_folder=val_img_folder,
    transform=val_transform,
    use_bbox=True
)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
print(f"Number of training samples:   {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")


Number of training samples:   6461
Number of validation samples: 1300


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model = LargerShapeTextureNet(num_classes=13)
model = model.to(device)

Using device: cpu


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
NUM_EPOCHS = 20  # Adjust as needed

def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

train_history = []
val_history = []

for epoch in range(NUM_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    train_history.append((train_loss, train_acc))
    val_history.append((val_loss, val_acc))
    
    epoch_time = time.time() - start_time
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% "
          f"Time: {epoch_time:.2f}s")


Epoch [1/20] Train Loss: 2.5147 | Train Acc: 12.95% Val Loss: 2.4865 | Val Acc: 14.31% Time: 1710.45s
Epoch [2/20] Train Loss: 2.4423 | Train Acc: 16.55% Val Loss: 2.4702 | Val Acc: 15.15% Time: 1621.87s
Epoch [3/20] Train Loss: 2.3871 | Train Acc: 18.70% Val Loss: 2.4806 | Val Acc: 17.15% Time: 1795.25s
Epoch [4/20] Train Loss: 2.3420 | Train Acc: 19.63% Val Loss: 2.4092 | Val Acc: 16.31% Time: 1597.40s
Epoch [5/20] Train Loss: 2.2934 | Train Acc: 22.04% Val Loss: 2.3818 | Val Acc: 19.23% Time: 1582.01s
Epoch [6/20] Train Loss: 2.2535 | Train Acc: 23.79% Val Loss: 2.3045 | Val Acc: 23.15% Time: 1431.06s
Epoch [7/20] Train Loss: 2.2106 | Train Acc: 25.54% Val Loss: 2.4061 | Val Acc: 17.85% Time: 1545.03s
Epoch [8/20] Train Loss: 2.1655 | Train Acc: 27.22% Val Loss: 2.3059 | Val Acc: 24.15% Time: 1483.90s
Epoch [9/20] Train Loss: 2.1297 | Train Acc: 27.94% Val Loss: 2.2275 | Val Acc: 22.46% Time: 1421.44s
Epoch [10/20] Train Loss: 2.0963 | Train Acc: 29.39% Val Loss: 2.1837 | Val Acc: 2